Imports

In [ ]:
from openai import OpenAI

Initialize an instance of the `OpenAI` class, which is used to interact with OpenAI's API. This instance, referred to as `client`, will be the primary object through which we send requests and receive responses from the OpenAI service. Ensure that the OpenAI library is imported and configured with your API key before this step.

In [2]:
client = OpenAI()

Creating an Assistant

In [3]:
assistant = client.beta.assistants.create(
    name="AI-PenTest-Tool",
    instructions="""
                You are a Cyber Security Specialist with a vast knowledge of all fields of Cyber Security. 
                You will only respond to the query with the appropriate commands needed for the pentest. 
                If a code is required, respond with just the code snippet.
                You do not require to explain.
                Respond with the COMMANDS ONLY.
                1. A IP will be provided. Provide a steps for reconnaissance.
                2. Based on the output given, Provide commands to continue.
                """,
    tools=[{"type": "code_interpreter"}],
    model="gpt-4-turbo-preview"
)

The line `thread = client.beta.threads.create()` is used to create a new conversation thread with the OpenAI API. This command initializes a thread through the `client` object, which must already be configured with OpenAI API access. The `beta.threads.create` method specifically creates a new thread, providing a unique context for a series of interactions or a conversation with the AI model. This is particularly useful for maintaining continuity in an ongoing dialogue or a series of related queries.


In [4]:
thread = client.beta.threads.create()

The code `message = client.beta.threads.messages.create(...)` is used to send a message to the OpenAI API within a previously created thread. The `thread_id` parameter specifies the unique ID of the thread to which this message belongs, ensuring continuity in the conversation. The `role` is set to "user" to indicate that this message is from the user's perspective. The `content` parameter contains the specific query or instruction for the AI, such as requesting actions or information related to a specified task. This approach is used for interactive and context-aware conversations with the AI model.


In [5]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I want to pen test this machine 192.168.1.205"
)

List messages in a thread

In [14]:
print(client.beta.threads.messages.list(thread_id=thread.id))

SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_sKNehFjHzNyXsK2M1KXjHPy3', assistant_id='asst_UZfJM3XaOwkEuCPZr8Bx06XD', content=[MessageContentText(text=Text(annotations=[], value='1. Ping to check if the target is up:\n   ```\n   ping 192.168.1.205\n   ```\n\n2. Use Nmap to scan for open ports and services:\n   ```\n   nmap -sV -Pn 192.168.1.205\n   ```\n\n3. Perform OS detection with Nmap:\n   ```\n   nmap -O 192.168.1.205\n   ``` \n\n4. Check for vulnerabilities:\n   ```\n   nmap --script=vuln 192.168.1.205\n   ```'), type='text')], created_at=1706354787, file_ids=[], metadata={}, object='thread.message', role='assistant', run_id='run_9f9mgrjjrq6QeXO59yEe5MvA', thread_id='thread_T6oFAm6E5l5wcV9BmY6yPY8P'), ThreadMessage(id='msg_jrRBsiDRB9qn5Cy4U6OAWXsq', assistant_id=None, content=[MessageContentText(text=Text(annotations=[], value='I want to pen test this machine 192.168.1.205'), type='text')], created_at=1706354639, file_ids=[], metadata={}, object='thread.message', rol

For the Assistant to respond to the user message, you need to create a Run. This makes the Assistant read the Thread and decide whether to call tools (if they are enabled) or simply use the model to best answer the query.

In [9]:
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id,
)

In [13]:
# Example of checking the run status
run_status = client.beta.threads.runs.retrieve(run_id=run.id,thread_id=thread.id)
print(run_status)

Run(id='run_9f9mgrjjrq6QeXO59yEe5MvA', assistant_id='asst_UZfJM3XaOwkEuCPZr8Bx06XD', cancelled_at=None, completed_at=1706354791, created_at=1706354786, expires_at=None, failed_at=None, file_ids=[], instructions='\n                You are a Cyber Security Specialist with a vast knowledge of all fields of Cyber Security. \n                You will only respond to the query with the appropriate commands needed for the pentest. \n                If a code is required, respond with just the code snippet.\n                You do not require to explain.\n                Respond with the COMMANDS ONLY.\n                1. A IP will be provided. Provide a steps for reconnaissance.\n                2. Based on the output given, Provide commands to continue.\n                ', last_error=None, metadata={}, model='gpt-4-turbo-preview', object='thread.run', required_action=None, started_at=1706354786, status='completed', thread_id='thread_T6oFAm6E5l5wcV9BmY6yPY8P', tools=[ToolAssistantToolsCode(ty